In [4]:
import os
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np

In [5]:


class CustomDataset(Dataset):
    def __init__(self, image_folder, label_folder, transform=None, threshold = 0):
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.threshold = threshold
        self.transform = transform
        self.image_filenames = sorted([f for f in os.listdir(image_folder) if f.endswith(".jpg")])
        self.label_filenames = [f.replace(".jpg", "_mask.jpg") for f in self.image_filenames]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_filename = self.image_filenames[idx]
        label_filename = self.label_filenames[idx]

        image_path = os.path.join(self.image_folder, image_filename)
        label_path = os.path.join(self.label_folder, label_filename)

        image = Image.open(image_path).convert("RGB")
        label = Image.open(label_path).convert("L")
        
        

        if self.transform:
            image = self.transform(image)
            label = self.transform(label)
        
        label = np.array(label)
        label = (label > self.threshold).astype(np.float32)
        

        return image, label

In [6]:
transform = transforms.Compose([
#     transforms.Resize((448, 448)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
])

In [7]:
image_folder = "/home/macula/SMATousi/Gullies/ground_truth/leaf_vein_seg/sorted_data/images_448/"
label_folder = "/home/macula/SMATousi/Gullies/ground_truth/leaf_vein_seg/sorted_data/labels_448/"

In [8]:
custom_dataset = CustomDataset(image_folder, label_folder, transform, threshold=0)

In [9]:
batch_size = 32
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=False)

In [10]:
image, label = next(iter(data_loader))

In [11]:
label.shape

torch.Size([32, 1, 448, 448])

In [30]:
Image.open("/home/macula/SMATousi/Gullies/ground_truth/leaf_vein_seg/sorted_data/labels/1_Walnut_labels5_0000_图层 3.jpg").convert("RGB")





FileNotFoundError: [Errno 2] No such file or directory: '/home/macula/SMATousi/Gullies/ground_truth/leaf_vein_seg/sorted_data/labels/1_Walnut_labels5_0003_背景 3.jpg'

In [68]:
file_directory = "/home/macula/SMATousi/Gullies/ground_truth/leaf_vein_seg/sorted_data/labels/"

for old_file_name in os.listdir(file_directory):
    
#     print(old_file_name)
#     break
    # Split the filename by the "_" character
    parts = old_file_name.split("_")

    # Join the parts except the one containing "图层" to create the new filename
    new_filename = "_".join([part for part in parts if "000" not in part])
#     print(new_filename)
#     break
    # Get the directory path of the file
    # Replace with the actual directory path

    # Create the full path for both old and new filenames
    old_filepath = os.path.join(file_directory, old_file_name)
    new_filepath = os.path.join(file_directory, new_filename + ".jpg")
    
#     print(old_filepath)
#     print(new_filepath)
#     break

    # Rename the file
    os.rename(old_filepath, new_filepath)

In [33]:
os.rename("/home/macula/SMATousi/Gullies/ground_truth/leaf_vein_seg/sorted_data/labels/1_Walnut_labels5_0000", 
          "/home/macula/SMATousi/Gullies/ground_truth/leaf_vein_seg/sorted_data/labels/1_Walnut_labels5_0000.jpg")

In [69]:
file_directory = "/home/macula/SMATousi/Gullies/ground_truth/leaf_vein_seg/sorted_data/labels/"

for old_file_name in os.listdir(file_directory):
    
    if old_file_name.endswith(".jpg.jpg"):
        
        parts = old_file_name.split(".")
        
        new_file_name = parts[0]
        
        old_filepath = os.path.join(file_directory, old_file_name)
        new_filepath = os.path.join(file_directory, new_file_name + ".jpg")
        
        os.rename(old_filepath, new_filepath)
#         print("FUCK")

In [3]:
from PIL import Image
import os
from tqdm import tqdm

# Input and output directories
input_dir = "../sorted_data/labels/"
output_dir = "../sorted_data/labels_448/"
output_size = (448, 448)  # Specify the desired output size (width, height)

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# List all files in the input directory
file_list = os.listdir(input_dir)

for filename in tqdm(file_list):
    # Check if the file is an image (you can add more extensions if needed)
    if filename.endswith((".jpg", ".jpeg", ".png", ".gif")):
        try:
            # Open the image using Pillow
            img = Image.open(os.path.join(input_dir, filename))

            # Resize the image
            img = img.resize(output_size, Image.ANTIALIAS)

            # Save the resized image to the output directory
            img.save(os.path.join(output_dir, filename))
#             print(f"Resized and saved: {filename}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

 64%|████████████████████████████████                  | 3038/4746 [12:45<07:06,  4.01it/s]

Error processing _mask.jpg: [Errno 21] Is a directory: '../sorted_data/labels/_mask.jpg'


100%|██████████████████████████████████████████████████| 4746/4746 [19:57<00:00,  3.96it/s]
